## Build a semantic search engine

In [1]:
import os
from dotenv import load_dotenv

# Loading the .env file. Using full path to use it in the interactive shell
load_dotenv("C:\\Users\\MarcusChiri\\Dropbox\\Repositories\\Langchain\\.env")

# Loarding the environment variables
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"

## Documents and Document Loaders
LangChain implements a Document abstraction, which is intended to represent a unit of text and associated metadata. It has three attributes:

page_content: a string representing the content;
metadata: a dict containing arbitrary metadata;
id: (optional) a string identifier for the document.
The metadata attribute can capture information about the source of the document, its relationship to other documents, and other information. Note that an individual Document object often represents a chunk of a larger document.

We can generate sample documents when desired:

In [2]:
from langchain_core.documents import Document

documents = [
    Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"source": "mammal-pets-doc"},
    ),
]

In [3]:
from langchain_community.document_loaders import PyPDFLoader

file_path = "nke-10k-2023.pdf"
loader = PyPDFLoader(file_path)

docs = loader.load()

print(len(docs))

107


PyPDFLoader loads one Document object per PDF page. For each, we can easily access:

The string content of the page;
Metadata containing the file name and page number.

In [4]:
print(f"{docs[0].page_content[:200]}\n")
print(docs[0].metadata)

Table of Contents
UNITED STATES
SECURITIES AND EXCHANGE COMMISSION
Washington, D.C. 20549
FORM 10-K
(Mark One)
☑  ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(D) OF THE SECURITIES EXCHANGE ACT OF 1934
F

{'producer': 'EDGRpdf Service w/ EO.Pdf 22.0.40.0', 'creator': 'EDGAR Filing HTML Converter', 'creationdate': '2023-07-20T16:22:00-04:00', 'title': '0000320187-23-000039', 'author': 'EDGAR Online, a division of Donnelley Financial Solutions', 'subject': 'Form 10-K filed on 2023-07-20 for the period ending 2023-05-31', 'keywords': '0000320187-23-000039; ; 10-K', 'moddate': '2023-07-20T16:22:08-04:00', 'source': 'nke-10k-2023.pdf', 'total_pages': 107, 'page': 0, 'page_label': '1'}


## Splitting
Splitting tries to get more nuanced semantic meaning from documents. Langchain has builtin [text splitters](https://python.langchain.com/docs/concepts/text_splitters/) to do that.

They recommend [RecursiveCharacterTextSplitter](https://python.langchain.com/docs/how_to/recursive_text_splitter/) for generic use cases. This tool recognises line breaks and page breaks as natural ends for splitters.

In [26]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

len(all_splits)

516

In [ ]:
all_splits[15].dict()

## Embedding
Introduces the concept of embedding, which is to convert text into its numeric vector semantic representation.

In [31]:
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

In [ ]:
# each vector has 3k dimensions lol
vector_1 = embeddings.embed_query(all_splits[0].page_content)
vector_2 = embeddings.embed_query(all_splits[1].page_content)

Generated vectors of length 3072

[0.009298405610024929, -0.01608305238187313, 0.00028412663959898055, 0.0064094592817127705, 0.020547788590192795, -0.03926966339349747, -0.007359934970736504, 0.04102053865790367, -0.008072791621088982, 0.05998003110289574]


In [35]:

assert len(vector_1) == len(vector_2)
print(f"Generated vectors of length {len(vector_1)}\n")
print(vector_1[:10])

Generated vectors of length 3072

[0.009298405610024929, -0.01608305238187313, 0.00028412663959898055, 0.0064094592817127705, 0.020547788590192795, -0.03926966339349747, -0.007359934970736504, 0.04102053865790367, -0.008072791621088982, 0.05998003110289574]


## Vector Store
Vector store objects contain methods for adding text and Document objects to the store, and querying them using various similarity methods. There’s plenty of integrations with diverse stores. It can also be done with Postgres, but also in memory

In [ ]:
# It could be done in memory:
# from langchain_core.vectorstores import InMemoryVectorStore

# vector_store = InMemoryVectorStore(embeddings)
# ids = vector_store.add_documents(documents=all_splits)

If you need a quick refresher on Postgres implementation, [check this guide](https://www.notion.so/PostgreSQL-in-Python-1e1e007475598028af1beac16cada3df?pvs=21).

In [43]:
# pip install -qU langchain-postgres
from langchain_postgres import PGVector

from dotenv import load_dotenv
load_dotenv()

vector_store = PGVector(
    embeddings=embeddings,
    collection_name="my_docs",
    connection=os.getenv("PSQL_DB_URL")
)

Having instantiated our vector store, we can now index the documents:

In [44]:
ids = vector_store.add_documents(documents=all_splits)

Note that most vector store implementations will allow you to connect to an existing vector store-- e.g., by providing a client, index name, or other information. See the documentation for a specific integration for more detail.

Once we've instantiated a VectorStore that contains documents, we can query it. VectorStore includes methods for querying:
- Synchronously and asynchronously;
- By string query and by vector;
- With and without returning similarity scores;
- By similarity and maximum marginal relevance (to balance similarity with query to diversity in retrieved results).
- The methods will generally include a list of Document objects in their outputs.

## Usage
Embeddings typically represent text as a "dense" vector such that texts with similar meanings are geometrically close. This lets us retrieve relevant information just by passing in a question, without knowledge of any specific key-terms used in the document.

In [45]:
results = vector_store.similarity_search(
    "How many distribution centers does Nike have in the US?"
)

print(results[0])

page_content='operations. We also lease an office complex in Shanghai, China, our headquarters for our Greater China geography, occupied by employees focused on implementing our
wholesale, NIKE Direct and merchandising strategies in the region, among other functions.
In the United States, NIKE has eight significant distribution centers. Five are located in or near Memphis, Tennessee, two of which are owned and three of which are
leased. Two other distribution centers, one located in Indianapolis, Indiana and one located in Dayton, Tennessee, are leased and operated by third-party logistics
providers. One distribution center for Converse is located in Ontario, California, which is leased. NIKE has a number of distribution facilities outside the United States,
some of which are leased and operated by third-party logistics providers. The most significant distribution facilities outside the United States are located in Laakdal,' metadata={'page': 26, 'title': '0000320187-23-000039', 'autho

In [51]:
# this returns a list of Document objects, which are the most similar to the query
# The first one is the most similar, and the last one is the least similar.
# The similarity search is based on the embeddings of the documents and the query.
results

[Document(id='cbcecd54-16b9-4fde-a1b2-d785ea50ce15', metadata={'page': 26, 'title': '0000320187-23-000039', 'author': 'EDGAR Online, a division of Donnelley Financial Solutions', 'source': 'nke-10k-2023.pdf', 'creator': 'EDGAR Filing HTML Converter', 'moddate': '2023-07-20T16:22:08-04:00', 'subject': 'Form 10-K filed on 2023-07-20 for the period ending 2023-05-31', 'keywords': '0000320187-23-000039; ; 10-K', 'producer': 'EDGRpdf Service w/ EO.Pdf 22.0.40.0', 'page_label': '27', 'start_index': 804, 'total_pages': 107, 'creationdate': '2023-07-20T16:22:00-04:00'}, page_content='operations. We also lease an office complex in Shanghai, China, our headquarters for our Greater China geography, occupied by employees focused on implementing our\nwholesale, NIKE Direct and merchandising strategies in the region, among other functions.\nIn the United States, NIKE has eight significant distribution centers. Five are located in or near Memphis, Tennessee, two of which are owned and three of which 

Async query:

In [ ]:
# this is generating an error, but I won't fix it now, because I don't need it
results = await vector_store.asimilarity_search("When was Nike incorporated?")

print(results[0])

Return scores:

In [ ]:
# Note that providers implement different scores; the score here
# is a distance metric that varies inversely with similarity.

# notice the "with_score" method, which returns a list of tuples (Document, score)
results = vector_store.similarity_search_with_score(
    "What was Nike's revenue in 2023?")
doc, score = results[0]
print(f"Score: {score}\n")
print(doc)

Score: 0.3118425402594367

page_content='Table of Contents
FISCAL 2023 NIKE BRAND REVENUE HIGHLIGHTSThe following tables present NIKE Brand revenues disaggregated by reportable operating segment, distribution channel and major product line:
FISCAL 2023 COMPARED TO FISCAL 2022
• NIKE, Inc. Revenues were $51.2 billion in fiscal 2023, which increased 10% and 16% compared to fiscal 2022 on a reported and currency-neutral basis, respectively.
The increase was due to higher revenues in North America, Europe, Middle East & Africa ("EMEA"), APLA and Greater China, which contributed approximately 7, 6,
2 and 1 percentage points to NIKE, Inc. Revenues, respectively.
• NIKE Brand revenues, which represented over 90% of NIKE, Inc. Revenues, increased 10% and 16% on a reported and currency-neutral basis, respectively. This
increase was primarily due to higher revenues in Men's, the Jordan Brand, Women's and Kids' which grew 17%, 35%,11% and 10%, respectively, on a wholesale
equivalent basis.' metad

In [ ]:
print([score for doc, score in results])

[0.3118425402594367, 0.314993679523468, 0.3372749887272697, 0.34475127889964785]


In [ ]:
print((results[3][0]))

It can also search by vector instead of text. It does this by first embedding the query:

In [ ]:
embedding = embeddings.embed_query("How were Nike's margins impacted in 2023?")

results = vector_store.similarity_search_by_vector(embedding)
print(results[0])

## Retrievers
LangChain `VectorStore` objects do not belong to subclass [Runnable](https://python.langchain.com/api_reference/core/index.html#langchain-core-runnables). LangChain [Retrievers](https://python.langchain.com/api_reference/core/index.html#langchain-core-retrievers) are Runnables, so they implement a standard set of methods (e.g., synchronous and asynchronous `invoke` and `batch` operations). Although we can construct retrievers from vector stores, retrievers can interface with non-vector store sources of data, as well (such as external APIs).

We can create a simple version of this ourselves, without subclassing `Retriever`. If we choose what method we wish to use to retrieve documents, we can create a runnable easily. Below we will build one around the `similarity_search` method:

In [ ]:
from typing import List

from langchain_core.documents import Document
from langchain_core.runnables import chain


@chain
def retriever(query: str) -> List[Document]:
    return vector_store.similarity_search(query, k=1)


# for each query, it returns the most similar document.
# To return more documents, change the k parameter in the similarity_search method.
retriever.batch(
    [
        "How many distribution centers does Nike have in the US?",
        "When was Nike incorporated?",
    ],
)

[[Document(id='cbcecd54-16b9-4fde-a1b2-d785ea50ce15', metadata={'page': 26, 'title': '0000320187-23-000039', 'author': 'EDGAR Online, a division of Donnelley Financial Solutions', 'source': 'nke-10k-2023.pdf', 'creator': 'EDGAR Filing HTML Converter', 'moddate': '2023-07-20T16:22:08-04:00', 'subject': 'Form 10-K filed on 2023-07-20 for the period ending 2023-05-31', 'keywords': '0000320187-23-000039; ; 10-K', 'producer': 'EDGRpdf Service w/ EO.Pdf 22.0.40.0', 'page_label': '27', 'start_index': 804, 'total_pages': 107, 'creationdate': '2023-07-20T16:22:00-04:00'}, page_content='operations. We also lease an office complex in Shanghai, China, our headquarters for our Greater China geography, occupied by employees focused on implementing our\nwholesale, NIKE Direct and merchandising strategies in the region, among other functions.\nIn the United States, NIKE has eight significant distribution centers. Five are located in or near Memphis, Tennessee, two of which are owned and three of which

Vectorstores implement an as_retriever method that will generate a Retriever, specifically a VectorStoreRetriever. These retrievers include specific search_type and search_kwargs attributes that identify what methods of the underlying vector store to call, and how to parameterize them. For instance, we can replicate the above with the following:

In [ ]:
retriever = vector_store.as_retriever(
    search_type="similarity",
    # take a look at search_kwargs, it can receive lots of cool parameters
    search_kwargs={"k": 1},
)

retriever.batch(
    [
        "How many distribution centers does Nike have in the US?",
        "When was Nike incorporated?",
    ],
)

[[Document(id='cbcecd54-16b9-4fde-a1b2-d785ea50ce15', metadata={'page': 26, 'title': '0000320187-23-000039', 'author': 'EDGAR Online, a division of Donnelley Financial Solutions', 'source': 'nke-10k-2023.pdf', 'creator': 'EDGAR Filing HTML Converter', 'moddate': '2023-07-20T16:22:08-04:00', 'subject': 'Form 10-K filed on 2023-07-20 for the period ending 2023-05-31', 'keywords': '0000320187-23-000039; ; 10-K', 'producer': 'EDGRpdf Service w/ EO.Pdf 22.0.40.0', 'page_label': '27', 'start_index': 804, 'total_pages': 107, 'creationdate': '2023-07-20T16:22:00-04:00'}, page_content='operations. We also lease an office complex in Shanghai, China, our headquarters for our Greater China geography, occupied by employees focused on implementing our\nwholesale, NIKE Direct and merchandising strategies in the region, among other functions.\nIn the United States, NIKE has eight significant distribution centers. Five are located in or near Memphis, Tennessee, two of which are owned and three of which

VectorStoreRetriever supports search types of "similarity" (default), "mmr" (maximum marginal relevance, described above), and "similarity_score_threshold". We can use the latter to threshold documents output by the retriever by similarity score.

Retrievers can easily be incorporated into more complex applications, such as retrieval-augmented generation (RAG) applications that combine a given question with retrieved context into a prompt for a LLM. To learn more about building such an application, check out the RAG tutorial tutorial.